In [4]:
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [9]:
tennis_data <- read_csv("data/player_stats.csv")

New names:
• `` -> `...1`
Rows: 500 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (25): Age, Country, Plays, Wikipedia, Current Rank, Best Rank, Name, Bac...
dbl (13): ...1, Turned Pro, Seasons, Titles, Best Season, Retired, Masters, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [14]:
colnames(tennis_data)

[1] "...1"             "Age"              "Country"          "Plays"           
 [5] "Wikipedia"        "Current Rank"     "Best Rank"        "Name"            
 [9] "Backhand"         "Prize Money"      "Height"           "Favorite Surface"
[13] "Turned Pro"       "Seasons"          "Active"           "Current Elo Rank"
[17] "Best Elo Rank"    "Peak Elo Rating"  "Last Appearance"  "Titles"          
[21] "GOAT Rank"        "Best Season"      "Retired"          "Masters"         
[25] "Birthplace"       "Residence"        "Weight"           "Coach"           
[29] "Facebook"         "Twitter"          "Nicknames"        "Grand Slams"     
[33] "Davis Cups"       "Web Site"         "Team Cups"        "Olympics"        
[37] "Weeks at No. 1"   "Tour Finals"

In [15]:
filtered_tennis_data <- tennis_data %>% filter(Age.)

ERROR: [1m[33mError[39m in [1m[1m`filter()`:[22m
[1m[22m[33m![39m Problem while computing `..1 = Age.`.
[1mCaused by error in [1m[1m`mask$eval_all_filter()`:[22m
[33m![39m object 'Age.' not found
